In [1]:
import numpy as np
import IPython
import IPython.display as ipd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as color
import soundfile
import scipy.signal as sig
import sklearn
import iracema
import math
import os
from os import listdir
import scipy.signal as sig
from os.path import isfile, join

In [2]:
X = pd.read_csv('nn_ele_hb_00-24hr_TrainingSet_v2.txt', sep="\t", header=0) 


list_of_dataframes=[]

for file_path in listdir('datafiles'): 
    
    samples,sampling_rate=soundfile.read('datafiles/'+file_path)

    num_rows= np.shape(X[X['filename']==file_path])[0]
    ref_table=X[X['filename']==file_path]
    
    
    t_start=ref_table[ref_table.columns[2]][0:num_rows+1]
    t_end=ref_table[ref_table.columns[2]][0:num_rows+1]+ref_table[ref_table.columns[4]][0:num_rows+1]
    


    window_size, hop_size = int(sampling_rate*2), 1024 #need to change

    data=[]

    for i,j in zip(t_start.values.tolist(),t_end.values.tolist()):
        
        #s = samples[math.floor(i)*sampling_rate:math.ceil(j)*sampling_rate] # for filtering
        #[b,a] = sig.butter(4,[10/sampling_rate, 100/sampling_rate],'bandpass','ba') 
        #data1 = sig.filtfilt(b, a, s)
        
        
    
        audio = iracema.Audio(samples[math.floor(i)*sampling_rate:math.ceil(j)*sampling_rate], sampling_rate)
        
        #audio = iracema.Audio(data1, sampling_rate) uncomment if you want to filter and have uncommented above three lines of code
        
        fft = iracema.spectral.fft(audio, window_size, hop_size)
        pitch = iracema.pitch.hps(fft,minf0=10, maxf0=100)
        harmonics = iracema.harmonics.extract(fft, pitch)
    
        # extract some features
        data.append([iracema.features.spectral_flux(fft).data.mean(),
                 iracema.features.spectral_flatness(fft).data.mean(),
                 iracema.features.spectral_centroid(fft).data.mean(),
                 iracema.features.hfc(fft).data.mean(),
                 iracema.features.spectral_entropy(fft).data.mean(),
                 iracema.features.spectral_energy(fft).data.mean(),
                 pitch.data.mean(),
                 iracema.features.noisiness(fft,harmonics['magnitude']).data.mean(),
                 iracema.features.peak_envelope(audio, window_size, hop_size).data.mean(),
                 iracema.features.rms(audio, window_size, hop_size).data.mean(),
                 iracema.features.zcr(audio, window_size, hop_size).data.mean(),
                 iracema.features.harmonic_energy(harmonics['magnitude']).data.mean()
                ]
               )
    
    list_of_dataframes.append(ref_table.reset_index().drop('index',axis=1).join(pd.DataFrame(data)))    
    data=[]

In [3]:
df = pd.concat(list_of_dataframes)

In [4]:
df.columns = ['Selection',
  'High Freq (Hz)',
  'File Offset (s)',
  'filename',
  'duration',
  'marginals',
  'spectral_flux_avg',
  'spectral_flatness_avg',
  'spectral_centroid_avg',
  'high_frequency_content_avg',
  'spectral_entropy_avg',
  'spectral_energy_avg',
  'pitch_avg',
  'noisiness_avg',
  'peak_envelope_avg',
  'rms_avg',
  'zcr_avg',
  'harmonic_energy_avg']

In [5]:
df.head()

,Selection,High Freq (Hz),File Offset (s),filename,duration,marginals,spectral_flux_avg,spectral_flatness_avg,spectral_centroid_avg,high_frequency_content_avg,spectral_entropy_avg,spectral_energy_avg,pitch_avg,noisiness_avg,peak_envelope_avg,rms_avg,zcr_avg,harmonic_energy_avg
0,1332,41.1,75690.835,nn04d_20180717_000000.wav,4.470,NaN,544.593322+0.000000j,-3.528925,1836.920962,3262.311459,0.297729,35260.841846,23.247863,0.998703,0.133464,0.041164,686.172727,19.702860
1,1333,40.0,75701.623,nn04d_20180717_000000.wav,6.022,NaN,582.591674+0.000000j,-3.449734,1810.448966,3576.093687,0.312143,37572.682124,25.489854,0.998372,0.226230,0.042951,900.912698,28.089712
2,1334,34.4,75715.352,nn04d_20180717_000000.wav,4.433,NaN,548.506670+0.000000j,-3.334850,1862.001548,3212.308031,0.299587,33906.389702,21.510404,0.998517,0.147171,0.040134,595.500000,26.847570
3,1335,34.4,75762.665,nn04d_20180717_000000.wav,5.357,NaN,523.751356+0.000000j,-3.630448,1832.615918,3043.409620,0.277098,34755.163252,21.272555,0.998291,0.126476,0.040692,524.492063,25.982743
0,1336,33.3,2972.771,nn04f_20180616_000000.wav,1.737,marginal,312.897239+0.000000j,-3.309776,1854.161600,1219.864332,0.409539,7602.226522,14.407814,0.996671,0.079414,0.019081,1153.484375,17.246007


* [Documentation](https://cegeme.github.io/iracema/source/iracema_features.html)
* [Paper](https://compmus.ime.usp.br/sbcm/2019/papers/sbcm-2019-3.pdf)
* [Github](https://github.com/cegeme/iracema)